In [62]:
# import requests
import pandas as pd

from airflow.models.dag import DAG
from airflow.operators.python import PythonOperator

from kraken_api import (
    fetch_ohlc_data,
    convert_to_json
)

from pandas_funcs import (
    convert_api_call_to_dataframe,
    select_columns
)

# from ge_funcs import ge_validation


In [63]:
TICKER_PAIR = "XXBTZUSD"
COLUMN_NAMES = ["date", "open", "high", "low", "close", "vwap", "volume", "count"]
RELEVANT_COLUMNS = ["date", "open", "high", "low", "close"]

In [64]:
data = fetch_ohlc_data(TICKER_PAIR, "1d")

In [65]:
json_data = convert_to_json(data, RELEVANT_COLUMNS)

In [66]:
json_data

'[{"date": "2023-02-07", "open": 22765.5, "high": 23347.7, "low": 22750.1, "close": 23253.2}, {"date": "2023-02-08", "open": 23253.3, "high": 23450.0, "low": 22675.5, "close": 22964.8}, {"date": "2023-02-09", "open": 22963.6, "high": 23008.2, "low": 21700.0, "close": 21792.0}, {"date": "2023-02-10", "open": 21792.0, "high": 21939.9, "low": 21475.0, "close": 21639.8}, {"date": "2023-02-11", "open": 21638.2, "high": 21909.8, "low": 21610.9, "close": 21868.2}, {"date": "2023-02-12", "open": 21868.2, "high": 22099.9, "low": 21640.1, "close": 21786.0}, {"date": "2023-02-13", "open": 21786.0, "high": 21902.2, "low": 21359.1, "close": 21791.2}, {"date": "2023-02-14", "open": 21790.5, "high": 22331.3, "low": 21505.0, "close": 22213.1}, {"date": "2023-02-15", "open": 22213.2, "high": 24366.4, "low": 22038.5, "close": 24334.5}, {"date": "2023-02-16", "open": 24340.9, "high": 25275.7, "low": 23500.0, "close": 23521.5}, {"date": "2023-02-17", "open": 23521.5, "high": 25010.5, "low": 23340.4, "clos

In [ ]:
from pydantic import BaseModel, Field, field_validator
import joblib
import json
import os

class DataModel(BaseModel):
    date: str = Field(unique=True)
    open: float
    high: float
    low: float
    close: float
    
    __values__ = {}
    
    def __init__(self, **data):
        super().__init__(**data)
        self.__values__[self.date] = self
    
    @field_validator("date")
    def validate_unique_date(cls, value, **kwargs):
        if value in cls.__values__:
            raise ValueError(f"Duplicate date: {value}")
        return value


with open("btc_data.json", 'r') as f:
    data = json.load(f)
    

error_messages = []
error_bool = False
for row in data:
    try:
        data_model = DataModel.model_validate(row)
    except ValueError as e:
        error = e.errors()[0]
        print(error)
        error_messages.append(error)
        error_bool = True

    
print(error_bool)

False
